In [60]:
from twisted.internet import asyncioreactor
asyncioreactor.install()

from twisted.python import log

import os

import httpx

import attr

import ipywidgets

import dateutil

from dateutil import parser, tz

import datetime

from twisted.internet import reactor, task

import functools

In [23]:
@attr.s(auto_attribs=True, frozen=True)
class Project:
    name: str
    pid: int
        
    @classmethod
    def from_api(cls, entry):
        return cls(name=entry["name"], pid=entry["id"])

In [5]:
with open(os.path.expanduser("~/.toggl-token")) as fpin:
    token = fpin.read().strip()

In [8]:
client = httpx.Client(auth=(token, "api_token"))

In [51]:
def get_current(client, now):
    res = client.get("https://api.track.toggl.com/api/v8/time_entries/current")
    data = res.json()["data"]
    description = data["description"]
    since = parser.isoparse(data["start"])
    duration = now - since
    minutes, seconds = divmod(duration, datetime.timedelta(minutes=1))
    seconds = int(seconds.total_seconds())
    return description, minutes, seconds

In [64]:
def update(label, client, get_now):
    description, minutes, seconds = get_current(client, get_now())
    label.value = f"{description} {minutes}:{seconds:02d}"

In [16]:
res = client.get("https://api.track.toggl.com/api/v8/workspaces")
[workspace] = res.json()
workspace_id = workspace["id"]

In [18]:
res = client.get(f"https://api.track.toggl.com/api/v8/workspaces/{workspace_id}/projects")

In [24]:
projects = list(map(Project.from_api, res.json()))

In [52]:
current = ipywidgets.Label("")

In [66]:
updater = task.LoopingCall(update, current, client, functools.partial(datetime.datetime.now, tz.UTC))
failed = updater.start(5)
failed.addErrback(log.err)

In [67]:
project_selection=ipywidgets.Combobox(
    options=[project.name for project in projects],
    description='Project',
    ensure_option=True,
    disabled=False
)

In [69]:
description = ipywidgets.Text()

In [86]:
def new_task(_ignored):
    for project in projects:
        if project.name == project_selection.value:
            pid = project.pid
    data = dict(time_entry=dict(
        description=description.value,
        pid=pid,
        created_with="Jupyter toggler"
    ))
    res = client.post("https://api.track.toggl.com/api/v8/time_entries/start", json=data)

In [87]:
start = ipywidgets.Button(description="start")
start.on_click(new_task)

In [88]:
box = ipywidgets.VBox(
    children=[current, description, project_selection, start]
)
box